# **HW2: Decision Tree** 
In *assignment 2*, you need to:

1.   Implement the decision tree in 3 steps (with the example data)


*   Step 1: calculate the entropy
*   Step 2: search for the best split
*   Step 3: build the decision tree

2.   Predict the patients' death (hospDIED) in the *MIMIC* dataset

Please fill in your **studentID** here.

In [ ]:
STUDENT_ID = '108060033'

# **1. Implement the Decision Tree**
In the first part, you need to implement the decision tree by completing the given funcitions.

Also, you need to run those functions with given input variables and save the output to the implementation csv file **[STUDENT_ID]_implementation.csv**.

Implement a binary decision tree to classify *Restaurant* dataset.

## Import Packages

Note: You **cannot** import any other packages in the first part (implementation)!

In [ ]:
import numpy as np
import pandas as pd
import random

## Load the Example Data
First, load the *Restaurant waiting* dataset: **data.csv**

In [ ]:
example = pd.read_csv('https://raw.githubusercontent.com/aubreyyy24/HW2_data/main/data.csv')
example

,Unnamed: 0,Alternate,Bar,Friday,Hungry,Patrons,Price,Raining,Reservation,Type,WaitEstimate,Wait
0,X1,T,F,F,T,Some,High,F,T,French,8,T
1,X2,T,F,F,T,Full,Low,F,F,Thai,40,F
2,X3,F,T,F,F,Some,Low,F,F,Burger,8,T
3,X4,T,F,T,T,Full,Low,F,F,Thai,12,T
4,X5,T,F,T,F,Full,High,F,T,French,70,F
5,X6,F,T,F,T,Some,Medium,T,T,Italian,3,T
6,X7,F,T,F,F,None,Low,T,F,Burger,7,F
7,X8,F,F,F,T,Some,Medium,T,T,Thai,6,T
8,X9,F,T,T,F,Full,Low,T,F,Burger,80,F
9,X10,T,T,T,T,Full,High,F,T,Italian,20,F


In [ ]:
example = example.drop(['Unnamed: 0'], axis=1)

In [ ]:
# change the string categorical to integer labels
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
OUTPUT = 'Wait'
example['Alternate'] = labelencoder.fit_transform(example['Alternate'])
example['Bar'] = labelencoder.fit_transform(example['Bar'])
example['Friday'] = labelencoder.fit_transform(example['Friday'])
example['Hungry'] = labelencoder.fit_transform(example['Hungry'])
example['Patrons'] = labelencoder.fit_transform(example['Patrons'])
example['Price'] = labelencoder.fit_transform(example['Price'])
example['Raining'] = labelencoder.fit_transform(example['Raining'])
example['Reservation'] = labelencoder.fit_transform(example['Reservation'])
example['Type'] = labelencoder.fit_transform(example['Type'])

example

,Alternate,Bar,Friday,Hungry,Patrons,Price,Raining,Reservation,Type,WaitEstimate,Wait
0,1,0,0,1,2,0,0,1,1,8,T
1,1,0,0,1,0,1,0,0,3,40,F
2,0,1,0,0,2,1,0,0,0,8,T
3,1,0,1,1,0,1,0,0,3,12,T
4,1,0,1,0,0,0,0,1,1,70,F
5,0,1,0,1,2,2,1,1,2,3,T
6,0,1,0,0,1,1,1,0,0,7,F
7,0,0,0,1,2,2,1,1,3,6,T
8,0,1,1,0,0,1,1,0,0,80,F
9,1,1,1,1,0,0,0,1,2,20,F


## Calculating the Entropy *(20%)*


In [ ]:
# split the data by given attribute and its threshold
def partition(data, column, threshold):
  """
  The *partition* function will split the input data into 2 branches.
    args:
    *   data(DataFrame): the input data
    *   column(str): the attribute(column name)
    *   threshold(float): the column's threshold for splitting the data
    returns:
    *   match_branch(DataFrame): the divided data that matches the assigned column's threshold
    *   false_branch(DataFrame): the divided data that doesn't match the assigned column's threshold
  """
  match = data[column] <= threshold
  Nmatch = data[column] > threshold

  match_branch = data[match]
  false_branch = data[Nmatch]
  match_branch = match_branch.drop(columns=[column])
  false_branch = false_branch.drop(columns=[column])
  return match_branch, false_branch

If we try *partition(example, 'Patrons', 1.5)*:

split example to 

match_left(Patrons<= 1.5) (Patrons != Some) and  
false_right(Patrons > 1.5) (Patrons == Some)

column: Patrons, threshold: 1.5



In [ ]:
match_left, false_right = partition(example, 'Patrons', 1.5)
print(match_left.shape)
print(false_right.shape)
false_right.head(4)

(8, 10)
(4, 10)


,Alternate,Bar,Friday,Hungry,Price,Raining,Reservation,Type,WaitEstimate,Wait
0,1,0,0,1,0,0,1,1,8,T
2,0,1,0,0,1,0,0,0,8,T
5,0,1,0,1,2,1,1,2,3,T
7,0,0,0,1,2,1,1,3,6,T


In [ ]:
def entropy(data):
  """
  The *entropy* function will calculate the entropy of the node(data)
  args:
  *    data(DataFrame): the data you're calculating for the entropy
  output:
  *    entropy(float): the node(data)'s entropy     
  """
  filter = data[OUTPUT] == "T"
  total = data.shape[0]
  p = data[filter].shape[0] / total
  if p == 1 or p == 0:
    entropy = 0
  else:
    entropy = (-p)* np.log2(p) - (1-p) * np.log2(1-p)

  return entropy

An example of the output from entropy (Use this to check if your output is correct): 

In [ ]:
match, false = partition(example, 'Hungry', 0.5)
entropy1 = entropy(match)
print(entropy1)

0.7219280948873623


In [ ]:
#implementation 1: calculate the entropy of 'false_right' data
ans_entropy = entropy(false)
print(ans_entropy)

0.863120568566631


## Find the Best Split *(20%)*

In [160]:
# search for the best attribute and the value(threshold) to split the data
def findBestSplit(data):
    """
    The *findBestSplit* function finds the best combination of attribute and value(with the largest reduction in entropy) to split the data.
    args:
    *   data(DataFrame): the data you try to split(build the decision tree)
    output:
    *   column_best(str): the attribute(column) split with the largest reduction in entropy
    *   value_best(float): the value(threshold) of the column_best attribute to split the data
    """
    all_entropy = 9999        #the overall entropy : save the value of the smallest entropy
    column_best = 'NONE'
    value_best = 0

    for idx, col in enumerate(data.columns, start = 0):
      tmp_ent = 0
      if col != OUTPUT:
        th = 0
        data.sort_values(by=[col],inplace = True)
        data_th = data.to_numpy()
        for i in range(0,data.shape[0]-1):
          if data_th[i][idx] != data_th[i + 1][idx]:
            tmp = (data_th[i][idx] + data_th[i + 1][idx])/2
            if tmp != th:
              match_1, false_1 = partition(data, col, tmp)
              if match_1.shape[0] == 0 or false_1.shape[0] == 0:
                tmp_ent = 1
              else:
                tmp_ent = (match_1.shape[0] * entropy(match_1) + false_1.shape[0] * entropy(false_1))/ data.shape[0]
              if tmp_ent < all_entropy:
                th = tmp
                all_entropy = tmp_ent
                column_best = col
                value_best = th  


    return column_best, value_best
  

An example of the output from findBestSplit (Use this to check if your output is correct):

In [161]:
column_best, value_best = findBestSplit(match)
print(column_best)
print(value_best)

Patrons
1.5


In [162]:
#implementation 2: Find the best split of the 'false' data
ans_column, ans_value = findBestSplit(false)
print(ans_column, ans_value)

WaitEstimate 16.0


## Decision Tree Building *(30%)*

Use the above functions to help building the decision tree





In [164]:
def AddTree(data, depth, features, thresholds):
  T_val = data[OUTPUT] == "T"
  T_val = data[T_val].shape[0]
  F_val = data.shape[0] - T_val
  if depth == 0:
    if T_val >= F_val:
      return "T"
    else:
      return "F"
  if T_val == data.shape[0]:
    return "T"
  if F_val == data.shape[0]:
    return "F"
  new_dict = {}
  column_best, value_best = findBestSplit(data)
  features.append(column_best)
  thresholds.append(value_best)
  match_left, false_right = partition(data, column_best, value_best)
  new_dict['%s <= %s'%(column_best,value_best)] = AddTree(match_left, depth - 1, features, thresholds)
  new_dict['%s > %s'%(column_best,value_best)] = AddTree(false_right, depth - 1, features, thresholds)
  return new_dict

In [163]:
def buildTree(df, depth):
  """
  The *buildTree* function builds the decision tree
  args:
  *     df(DataFrame): the data you want to apply the decision tree
  *     depth(int) : the depth of your tree
  output:
  *     decisionSubTree(dict): the decision tree structure including root, branch, leaf(with the attributes and thresholds)
  *     features(list): the features(attributes) name in the decision tree structure(from root to branch and leaf)
  *     thresholds(list): the corresponding thresholds for the features in the 'features' list
  """
  decisionSubTree = {}
  features = []
  thresholds = []
  if depth == 0:
    return decisionSubTree, features, thresholds
  
  decisionSubTree = AddTree(df, depth, features, thresholds)

  return decisionSubTree, features, thresholds
  

An example of the output from buildTree (Use this to check if your output is correct): 

In [165]:
tree, features, thresholds= buildTree(example, 2)
print(tree)
print(features)
print(thresholds)

{'Patrons <= 1.5': {'Hungry <= 0.5': 'F', 'Hungry > 0.5': 'T'}, 'Patrons > 1.5': 'T'}
['Patrons', 'Hungry']
[1.5, 0.5]


In [166]:
#implementation 3: decision tree building (depth=10)
ans_tree, ans_features, ans_thresholds = buildTree(example,10)
print(ans_tree)
print(ans_features)
print(ans_thresholds)

{'Patrons <= 1.5': {'Hungry <= 0.5': 'F', 'Hungry > 0.5': {'Friday <= 0.5': 'F', 'Friday > 0.5': {'Price <= 0.5': 'F', 'Price > 0.5': 'T'}}}, 'Patrons > 1.5': 'T'}
['Patrons', 'Hungry', 'Friday', 'Price']
[1.5, 0.5, 0.5, 0.5]


In [157]:
#save implementation csv
ans_path = STUDENT_ID + '_implementation.csv'

imp = []
imp.append(ans_entropy)
imp.append(ans_column)
imp.append(ans_value)
for i in range(len(ans_features)):
  imp.append(ans_features[i])
for m in range(len(ans_thresholds)):
  imp.append(ans_thresholds[m])
print(imp)
pd.DataFrame(imp).to_csv(ans_path, header = None, index = None)

[0.863120568566631, 'WaitEstimate', 16.0, 'Patrons', 'Hungry', 'Friday', 'Price', 1.5, 0.5, 0.5, 0.5]


# **2. Classification with the MIMIC Dataset**
In the second part, you need to classify **'hospDIED'(death)** in the MIMIC dataset.

Please put the classification result in a csv file. (**[STUDENTID]_prediction.csv**)

**Note:** Decision tree is recommended but not mandatory.

## Prediction (Performance) *(20%)*

The **y_test** (hospDIED) of this data is hidden, you need to use the x_test.csv to predict the y_test.

**Note:** You **can** now import the packages you need here!

Get the MIMIC data here!

Data Description: 

[Data Description](https://docs.google.com/spreadsheets/d/1pxqxQFhIcv_hrgWEtwhXE6zBVQ5ISa-13PIhvXMtWCY/edit?usp=sharing) (You can find the data description here.)

**Note:**
*   You can select the features you want to use.
*   You can use any ML models to predict the y_test.

  (However, there is a 10% bonus if you **visualize** the decision tree in this part)


[ref]: *Johnson, A. E. W., Pollard, T. J., Shen, L., Lehman, L. H., Feng, M., Ghassemi, M., Moody, B., Szolovits, P., Celi, L. A., & Mark, R. G. (2016). MIMIC-III, a freely accessible critical care database. Scientific Data, 3, 160035.*


In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn import tree

In [ ]:
#Read data
x_train = pd.read_csv('https://raw.githubusercontent.com/aubreyyy24/HW2_data/main/x_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/aubreyyy24/HW2_data/main/y_train.csv')
x_train = x_train.drop(columns="indextime")

Complete your model with validation:

In [ ]:
#Split the data into training and validation sets
# x_training, x_valid= np.split(x_train,[20000])
# y_training, y_valid = np.split(y_train,[20000])
#Build you model and evaluate it on your validation set
clf_entropy = DecisionTreeClassifier(criterion = "entropy", random_state = 20, max_depth = 4, min_samples_leaf = 20) 
clf_entropy.fit(x_train, y_train)
# clf_entropy.fit(x_training, y_training)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=4, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=20, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=20, splitter='best')

Make the final prediction from your model:

In [ ]:
x_test = pd.read_csv('https://raw.githubusercontent.com/aubreyyy24/HW2_data/main/x_test.csv')
x_test = x_test.drop(columns="indextime")
y_pred = clf_entropy.predict(x_test)
# f1_score(y_valid, y_pred)

To export your predcition as a CSV file and hand in the CSV on elearn

In [ ]:
output_path = STUDENT_ID + '_prediction.csv'

tree_test_pred = pd.DataFrame( {'subject_id': x_test.subject_id,
                  'prediction': y_pred } )
tree_test_pred.to_csv(output_path, index = False)

## Visualizing the Decision Tree *(10% bonus)*

**Note:** Save the visualization result image as **[STUDENT_ID]_visualization.png**

*   Your visualization image of the decision tree can contain **five** layers at most.


In [ ]:
#Decision Tree Visualization
fig = plt.figure(figsize=(250,200))
tree.plot_tree(clf_entropy, feature_names=x_train.columns, class_names="hospDIED", filled=True)
fig.savefig('108060033_visualization.png')

Output hidden; open in https://colab.research.google.com to view.

# Report *(10%)*

Report should be submitted as a pdf file! (**[STUDENT_ID]_report.pdf**)

*   List the top 3 splitting features and their thresholds of your model
*   Briefly describe how you build the decision tree
*   Describe if you apply any improvement on your decision tree model
*   If you preprocess the MIMIC data in the second part (selecting features...), describe the work and reasons
*   Summarize your work
*   Do not exceed 2 pages!






# Save the Code File
Please save your code and submit it as an ipynb file! (**[STUDENT_ID]_hw2.ipynb**)